In [1]:
import requests

use_real_puzzle_data = False # else use demo data
separator = '\n' if True else ','

github_site_user = "https://raw.githubusercontent.com/stephenhumphrey/"
repository_branch = "advent-of-code/main/"
advent_of_code_date = "2021/12/09/"
demo_data_locator = "demo-data.txt"
puzzle_data_locator = "puzzle-data.txt"

prefix = github_site_user + repository_branch + advent_of_code_date
demo_data_URL = prefix + demo_data_locator
puzzle_data_URL = prefix + puzzle_data_locator

print(demo_data_URL)
print(puzzle_data_URL)

https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/09/demo-data.txt
https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/09/puzzle-data.txt


In [2]:
def verbose_list(plist_string,max_elements = 5):
    # local version of a utility to pretty-print big arrays
    plist = eval(plist_string)
    if len(plist) <= max_elements:
        sub = f"{plist}"
    elif max_elements < 2:
        sub = f"[{plist[0]}, ...]"
    else:
        half_max = max_elements // 2
        sub = "["
        for v in plist[0:half_max]:
            sub += f"{v}, "
        sub += "... "
        for v in plist[-half_max:-1]:
            sub += f"{v}, "
        sub += f"{plist[-1]}]"
        
    print(f"len({plist_string})={len(plist)}, {plist_string}={sub}")

test_verbose = [1,2,3,4,5,6,7,8,9]
verbose_list("test_verbose")

len(test_verbose)=9, test_verbose=[1, 2, ... 8, 9]


In [3]:
def parse_data_string(ds):
    lines = [v for v in ds.split(separator) if len(v.strip()) > 0]
    #print(lines)
    
    sentinel = 99 # wrap the given map boundaries in sentinels of level 99
    data = [[sentinel]] 
    for line in lines:
        line_data = [int(v) for v in line]
        data.append([sentinel]+line_data+[sentinel])
    data.append([sentinel] * (len(line) + 2))
    data[0] = [sentinel] * (len(line) + 2)
    
    return data

print( parse_data_string("123\n456") )

[[99, 99, 99, 99, 99], [99, 1, 2, 3, 99], [99, 4, 5, 6, 99], [99, 99, 99, 99, 99]]


In [4]:
demo_data_string = requests.get(demo_data_URL).text
demo_data = parse_data_string( demo_data_string )

print(f"demo_data dimensions = ({len(demo_data[0])},{len(demo_data)})")
verbose_list("demo_data") # defaults to 5

demo_data dimensions = (12,7)
len(demo_data)=7, demo_data=[[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99], [99, 2, 1, 9, 9, 9, 4, 3, 2, 1, 0, 99], ... [99, 9, 8, 9, 9, 9, 6, 5, 6, 7, 8, 99], [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]]


In [5]:
puzzle_data_string = requests.get(puzzle_data_URL).text
puzzle_data = parse_data_string( puzzle_data_string )

print(f"puzzle_data dimensions = ({len(puzzle_data[0])},{len(puzzle_data)})")
verbose_list("puzzle_data",4)

puzzle_data dimensions = (102,102)
len(puzzle_data)=102, puzzle_data=[[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99], [99, 8, 6, 5, 6, 4, 5, 6, 7, 8, 9, 4, 3, 2, 1, 2, 9, 8, 7, 6, 5, 3, 2, 3, 5, 6, 7, 8, 9, 8, 9, 9, 9, 9, 8, 7, 5, 4, 3, 2, 0, 1, 4, 6, 7, 8, 9, 7, 6, 7, 8, 9, 3, 2, 1, 3, 4, 6, 8, 9, 5, 3, 2, 0, 5, 4, 5, 9, 3, 2, 3, 9, 9, 7, 6, 7, 8, 9, 0, 1, 9, 9, 7, 6, 5, 5, 4, 2, 3, 5, 6, 7, 7, 9, 7, 6, 5, 5, 6, 6, 7, 99], ... [99, 9, 9, 9, 9, 9, 9, 8, 9, 3, 1, 2, 3, 4, 5, 8, 9, 9, 9, 7, 5, 2, 3, 5, 6, 7, 8, 9, 4, 3, 2, 4, 5, 7, 8, 9, 9, 9, 8, 4, 3, 7, 5, 6, 7, 8, 9, 2, 3, 4, 5, 7, 9, 9, 9, 7, 8, 7, 6, 8, 9, 9, 9, 8, 7, 6, 5, 4, 

In [6]:
def is_localized_minimum(d,x,y):
    level = d[y][x]
    #print(f"({x},{y}) = {h}")
    for (dx, dy) in [(-1,0),(0,-1),(1,0),(0,1)]:
        if d[y + dy][x + dx] <= level:
            return False
    return True

print(f"{is_localized_minimum(demo_data,2,3)=}, should be False")
print(f"{is_localized_minimum(demo_data,2,1)=}, should be True")

is_localized_minimum(demo_data,2,3)=False, should be False
is_localized_minimum(demo_data,2,1)=True, should be True


In [7]:
data = puzzle_data if use_real_puzzle_data else demo_data

In [8]:
if not use_real_puzzle_data:
    for yrow in data:
        line = ""
        for xval in yrow:
            line += f"{xval:2}  "
        print( line )

99  99  99  99  99  99  99  99  99  99  99  99  
99   2   1   9   9   9   4   3   2   1   0  99  
99   3   9   8   7   8   9   4   9   2   1  99  
99   9   8   5   6   7   8   9   8   9   2  99  
99   8   7   6   7   8   9   6   7   8   9  99  
99   9   8   9   9   9   6   5   6   7   8  99  
99  99  99  99  99  99  99  99  99  99  99  99  


In [9]:
output = []
minima = []

for y in range( 1, len(data) - 1 ):
    for x in range( 1, len(data[0]) - 1 ):
        if is_localized_minimum( data, x, y ):
            minima.append( (x,y) )
            # print(f"minimum at ({x},{y}) --> {data[y][x]}")
            output.append( data[y][x] )

print(f"Answer to Day 8, Part 1: {sum(output) + len(output)}")

verbose_list("minima")

Answer to Day 8, Part 1: 15
len(minima)=4, minima=[(2, 1), (10, 1), (3, 3), (7, 5)]


In [10]:
def process_node(data,prior_nodes,x,y):
    
    home = (x,y)
    home_hash = hash(home)
    if home_hash in prior_nodes:
        return set([]) # empty set - this node is already processed
    
    bigset = prior_nodes | set([home_hash]) # add self node, now being processed
    home_level = data[y][x]
    
    for (dx,dy) in [(-1,0),(0,-1),(1,0),(0,1)]:
        neighbor_x = x + dx
        neighbor_y = y + dy
        neighbor_level = data[neighbor_y][neighbor_x]
        
        if neighbor_level < 9 and neighbor_level >= home_level:
            bigset |= process_node( data, bigset, neighbor_x, neighbor_y )
        
    return bigset - prior_nodes

print(f"{len(process_node(demo_data, set([]), 10, 1))=}, should be 9")

len(process_node(demo_data, set([]), 10, 1))=9, should be 9


In [11]:
sizes = []
for (x, y) in minima:
    empty_set = set([])
    basin_set = process_node( data, empty_set, x, y )
    #print(len(basin_set))
    sizes.append( len(basin_set) )

sizes.sort( reverse = True )
verbose_list( "sizes", 9 )

biggest = sizes[0:3]
multiple = biggest[0] * biggest[1] * biggest[2]

print( f"Answer to Day 8, Part 2: {multiple}" )

len(sizes)=4, sizes=[14, 9, 9, 3]
Answer to Day 8, Part 2: 1134
